# 2.1 - Format widget connections for LoRA finetuning

In [19]:
import os
import json
import numpy as np

In [5]:
connections = json.load(open(os.path.join(os.getcwd(), '..', 'data', 'orange_widgets_connections.json')))
len(connections)

29

In [3]:
all_widgets = list(set([i for x in [[c['widget']] + c['valid_inputs'] + c['invalid_inputs'] + c['valid_outputs'] + c['invalid_outputs'] for c in connections] for i in x]))
len(all_widgets)

88

    {
        "file": "orange3-addons/README.txt",
        "type": "QA",
        "question": "What is the purpose of the 'generate.py' script in the orange3-addons repository?",
        "answer": "The script generates a file named 'list', which contains package descriptions for official add-ons obtained through PyPI's JSON API."
    },
    {
        "file": "orange3-addons/README.txt",
        "type": "MCQ",
        "question": "Where must the file named 'list', generated by generate.py, be served for Orange add-ons?",
        "A": "https://pypi.org/simple",
        "B": "https://orange.biolab.si/addons/list",
        "C": "https://github.com/biolab/orange3",
        "D": "Localhost only",
        "correct": "B"
    },

In [32]:
questions_QA = []
valid_connections = []
invalid_connections = []

for widget in connections:

    FILE = "connections.json"

    # valid inputs
    for valid_input in widget['valid_inputs']:
        questions_QA.append({
            "file": FILE,
            "type": "QA",
            "question": f"Am I correct to say the {valid_input} widget is a valid input to the {widget['widget']} widget? {valid_input} --> {widget['widget']}",
            "answer": f"Yes. The {valid_input} widget is a valid input to the {widget['widget']} widget, notated as a connection {valid_input} --> {widget['widget']}."
        })
        questions_QA.append({
            "file": FILE,
            "type": "QA",
            "question": f"Am I correct to say the {valid_input} widget is an invalid input to the {widget['widget']} widget? {valid_input} -x-> {widget['widget']}",
            "answer": f"No. The {valid_input} widget is a valid input to the {widget['widget']} widget, notated as a connection {valid_input} --> {widget['widget']}."
        })

        valid_connections.append(f"{valid_input} --> {widget['widget']}")

    for invalid_input in widget['invalid_inputs']:
        questions_QA.append({
            "file": FILE,
            "type": "QA",
            "question": f"Am I correct to say the {invalid_input} widget is a valid input to the {widget['widget']} widget? {invalid_input} --> {widget['widget']}",
            "answer": f"No. The {invalid_input} widget is an invalid input to the {widget['widget']} widget, notated as a connection {invalid_input} -x-> {widget['widget']}."
        })
        questions_QA.append({
            "file": FILE,
            "type": "QA",
            "question": f"Am I correct to say the {invalid_input} widget is an invalid input to the {widget['widget']} widget? {invalid_input} -x-> {widget['widget']}",
            "answer": f"Yes. The {invalid_input} widget is an invalid input to the {widget['widget']} widget, notated as a connection {invalid_input} -x-> {widget['widget']}."
        })

        invalid_connections.append(f"{invalid_input} -x-> {widget['widget']}")
    
    for valid_output in widget['valid_outputs']:
        questions_QA.append({
            "file": FILE,
            "type": "QA",
            "question": f"Am I correct to say the {widget['widget']} widget is a valid input to the {valid_output} widget? {widget['widget']} --> {valid_output}",
            "answer": f"Yes. The {widget['widget']} widget is a valid input to the {valid_output} widget, notated as a connection {widget['widget']} --> {valid_output}."
        })
        questions_QA.append({
            "file": FILE,
            "type": "QA",
            "question": f"Am I correct to say the {widget['widget']} widget is an invalid input to the {valid_output} widget? {widget['widget']} -x-> {valid_output}",
            "answer": f"No. The {widget['widget']} widget is a valid input to the {valid_output} widget, notated as a connection {widget['widget']} --> {valid_output}."
        })

        valid_connections.append(f"{widget['widget']} --> {valid_output}")

    # invalid outputs
    for invalid_output in widget['invalid_outputs']:
        questions_QA.append({
            "file": FILE,
            "type": "QA",
            "question": f"Am I correct to say the {widget['widget']} widget is a valid input to the {invalid_output} widget? {widget['widget']} --> {invalid_output}",
            "answer": f"No. The {widget['widget']} widget is an invalid input to the {invalid_output} widget, notated as a connection {widget['widget']} -x-> {invalid_output}."
        })
        questions_QA.append({
            "file": FILE,
            "type": "QA",
            "question": f"Am I correct to say the {widget['widget']} widget is an invalid input to the {invalid_output} widget? {widget['widget']} -x-> {invalid_output}",
            "answer": f"Yes. The {widget['widget']} widget is an invalid input to the {invalid_output} widget, notated as a connection {widget['widget']} -x-> {invalid_output}."
        })

        invalid_connections.append(f"{widget['widget']} -x-> {invalid_output}")

print(f"Total questions: {len(questions_QA)}")
print(f"Valid connections: {len(valid_connections)}")
print(f"Invalid connections: {len(invalid_connections)}")



Total questions: 700
Valid connections: 206
Invalid connections: 144


In [38]:
np.random.seed(42)
questions_MCQ = []

for i in range(len(valid_connections)):

    all_valid = valid_connections.copy()
    target_connection = all_valid.pop(i)

    ## target connection is the only valid connection
    invalid_targets = invalid_connections.copy()
    np.random.shuffle(invalid_targets)
    invalid_targets = invalid_targets[:3]
    questions_MCQ.append({
        "file": "connections.json",
        "type": "MCQ-con",
        "question": f"Which of the following is a valid connection?",
        "A": target_connection,
        "B": invalid_targets[0].replace('-x->', '-->'),
        "C": invalid_targets[1].replace('-x->', '-->'),
        "D": invalid_targets[2].replace('-x->', '-->'),
        "correct": "A"
    })

    ## target connection is the only non-invalid connection
    invalid_targets = invalid_connections.copy()
    np.random.shuffle(invalid_targets)
    invalid_targets = invalid_targets[:3]
    questions_MCQ.append({
        "file": "connections.json",
        "type": "MCQ-con",
        "question": f"Which of the following is not an invalid connection?",
        "A": target_connection.replace('-->', '-x->'),
        "B": invalid_targets[0],
        "C": invalid_targets[1],
        "D": invalid_targets[2],
        "correct": "A"
    })

for i in range(len(invalid_connections)):

    all_invalid = invalid_connections.copy()
    target_connection = all_invalid.pop(i)

    ## target connection is the only invalid connection
    valid_targets = valid_connections.copy()
    np.random.shuffle(valid_targets)
    valid_targets = valid_targets[:3]
    questions_MCQ.append({
        "file": "connections.json",
        "type": "MCQ-con",
        "question": f"Which of the following is an invalid connection?",
        "A": target_connection,
        "B": valid_targets[0].replace('-->', '-x->'),
        "C": valid_targets[1].replace('-->', '-x->'),
        "D": valid_targets[2].replace('-->', '-x->'),
        "correct": "A"
    })

    ## target connection is the only valid connection
    valid_targets = valid_connections.copy()
    np.random.shuffle(valid_targets)
    valid_targets = valid_targets[:3]
    questions_MCQ.append({
        "file": "connections.json",
        "type": "MCQ-con",
        "question": f"Which of the following is not a valid connection?",
        "A": target_connection.replace('-x->', '-->'),
        "B": valid_targets[0],
        "C": valid_targets[1],
        "D": valid_targets[2],
        "correct": "A"
    })

print(f"Total MCQ questions: {len(questions_MCQ)}")

Total MCQ questions: 700


In [41]:
questions_MCQ[3]

{'file': 'connections.json',
 'type': 'MCQ-con',
 'question': 'Which of the following is not an invalid connection?',
 'A': 'File -x-> Scatter Plot',
 'B': 'Naive Bayes -x-> Hierarchical Clustering',
 'C': 'Distances -x-> Confusion Matrix',
 'D': 'Confusion Matrix -x-> Predictions',
 'correct': 'A'}

In [45]:
with open('../data/orange_connection_questions_MCQ.json', 'w') as f:
    json.dump(questions_MCQ, f, indent=4)

with open('../data/orange_connection_questions_QA.json', 'w') as f:
    json.dump(questions_QA, f, indent=4)

with open('../data/injected_tokens.json', 'w') as f:
    json.dump(all_widgets + ["-->", "-x->"], f, indent=4)
